In [1]:
import os
from pydub import AudioSegment, silence
import librosa
import numpy as np
import noisereduce as nr

# Klasör yolları
input_folder = "piyano_sarkilari"  # İndirilen şarkılar burada
output_folder = "temizlenmis_parcalarPiyano"  # Çıktılar buraya kaydedilecek
mfcc_folder = "mfcc_cikti_piyano"  # MFCC çıktılarını kaydedecek klasör

# Klasörleri oluştur (varsa hata vermez)
os.makedirs(output_folder, exist_ok=True)
os.makedirs(mfcc_folder, exist_ok=True)

# Sessizlik Temizleme Fonksiyonu
def remove_silence(audio_file, silence_thresh=-50, min_silence_len=500):
    """Verilen MP3 dosyasından sessiz bölümleri kaldırır."""
    audio = AudioSegment.from_mp3(audio_file)
    non_silent_chunks = silence.detect_nonsilent(audio, min_silence_len, silence_thresh)
    cleaned_audio = sum([audio[start:end] for start, end in non_silent_chunks])
    return cleaned_audio

# Gürültü Temizleme ve Normalizasyon Fonksiyonu
def process_audio(y, sr):
    # Gürültü Temizleme
    y_denoised = nr.reduce_noise(y=y, sr=sr)
    # Normalizasyon
    y_normalized = librosa.util.normalize(y_denoised)
    return y_normalized

# MFCC Çıkarma Fonksiyonu
def extract_mfcc(y, sr, n_mfcc=13):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs

# Şarkıları Parçalara Ayırma Fonksiyonu
def split_audio(audio, file_name, output_folder, segment_duration=3):
    """AudioSegment nesnesini segmentlere ayırır ve her segment için işlemler yapar."""
    total_duration = len(audio) / 1000  # milisaniyeden saniyeye

    for i in range(0, int(total_duration), segment_duration):
        segment = audio[i * 1000: (i + segment_duration) * 1000]
        
        # Segmenti geçici bir dosyaya kaydet
        segment_path = os.path.join(output_folder, f"{file_name}_segment_{i}.wav")
        segment.export(segment_path, format="wav")
        print(f"{segment_path} kaydedildi.")
        
        # Gürültü temizleme, normalizasyon ve MFCC çıkarma işlemleri
        y, sr = librosa.load(segment_path, sr=None)
        y_processed = process_audio(y, sr)
        mfcc_features = extract_mfcc(y_processed, sr)

        # MFCC sonuçlarını kaydet
        mfcc_file = os.path.join(mfcc_folder, f"{file_name}_segment_{i}_mfcc.npy")
        np.save(mfcc_file, mfcc_features)
        print(f"{mfcc_file} MFCC özellikleri kaydedildi.")

# Tüm şarkılar için işlemleri uygula
for file_name in os.listdir(input_folder):
    if file_name.endswith(".mp3"):
        file_path = os.path.join(input_folder, file_name)
        
        # Sessizliği Temizle
        print(f"{file_name} için sessizlik temizleniyor...")
        cleaned_audio = remove_silence(file_path)

        # Parçalara Ayır ve İşle
        print(f"{file_name} parçalara ayrılıyor ve işleniyor...")
        split_audio(cleaned_audio, file_name[:-4], output_folder, segment_duration=3)

print("Tüm şarkılar başarıyla işlendi.")

# Dosya adlarını yeniden adlandır
print("Dosya isimleri yeniden adlandırılıyor...")
if os.path.isdir(output_folder):
    mp3_dosyalari = [dosya for dosya in os.listdir(output_folder) if dosya.endswith(".wav")]
    for i, dosya in enumerate(mp3_dosyalari):
        yeni_dosya_adi = f"piyano_{i + 1}.wav"  # .wav uzantılı olarak yeniden adlandırma
        eski_dosya_yolu = os.path.join(output_folder, dosya)
        yeni_dosya_yolu = os.path.join(output_folder, yeni_dosya_adi)

        try:
            # Dosyayı yeniden adlandır
            os.rename(eski_dosya_yolu, yeni_dosya_yolu)
            print(f"{dosya} -> {yeni_dosya_adi} olarak yeniden adlandırıldı.")
        except Exception as e:
            print(f"Dosya adlandırılırken hata oluştu: {e}")
else:
    print("Belirtilen klasör bulunamadı.")


Ah Kavaklar - Sezen Aksu Cover - Piyano.mp3 için sessizlik temizleniyor...
Ah Kavaklar - Sezen Aksu Cover - Piyano.mp3 parçalara ayrılıyor ve işleniyor...
temizlenmis_parcalarPiyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_0.wav kaydedildi.
mfcc_cikti_piyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_0_mfcc.npy MFCC özellikleri kaydedildi.
temizlenmis_parcalarPiyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_3.wav kaydedildi.
mfcc_cikti_piyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_3_mfcc.npy MFCC özellikleri kaydedildi.
temizlenmis_parcalarPiyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_6.wav kaydedildi.
mfcc_cikti_piyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_6_mfcc.npy MFCC özellikleri kaydedildi.
temizlenmis_parcalarPiyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_9.wav kaydedildi.
mfcc_cikti_piyano\Ah Kavaklar - Sezen Aksu Cover - Piyano_segment_9_mfcc.npy MFCC özellikleri kaydedildi.
temizlenmis_parcalarPiyano\Ah Kavaklar - Sezen Ak